# **Durée de retour à l'emploi : phase exploratoire**

In [1]:
# nécessité d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
!pip install xlrd

In [2]:
!pip install openpyxl
# également demandé par Python pour lire Excel

In [3]:
import pandas as pd
import xlrd
import openpyxl
import numpy as np

## 1. Constitution de la BDD OPEN DATA

4 fichiers sont nécessaires à la constitution de cette df :
- 2 indicateurs de dynamisme du marché du travail : taux de chômage et indice de concentration de l'emploi
- 2 codes pour opérer les jointures : code INSEE de la commune (jointure avec les codes postaux) et de la Zone d'emploi (jointure avec les codes communes et le taux de chômage)

**Chargement du fichier de correspondance Code INSEE - Code postal**
- Source : La Poste sur Open Data

> *La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022*

- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [4]:
communes_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\laposte_hexasmal_4.xlsx")
communes_df.head(10)

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,01001,L ABERGEMENT CLEMENCIAT,1400,NaN,L ABERGEMENT CLEMENCIAT,"46.153721024,4.925850148"
1,01002,L ABERGEMENT DE VAREY,1640,NaN,L ABERGEMENT DE VAREY,"46.009605679,5.428087796"
2,01004,AMBERIEU EN BUGEY,1500,NaN,AMBERIEU EN BUGEY,"45.961048852,5.372275427"
3,01005,AMBERIEUX EN DOMBES,1330,NaN,AMBERIEUX EN DOMBES,"45.99616357,4.911967138"
4,01006,AMBLEON,1300,NaN,AMBLEON,"45.749886304,5.594584599"
5,01007,AMBRONAY,1500,NaN,AMBRONAY,"46.005690539,5.357748739"
6,01008,AMBUTRIX,1500,NaN,AMBUTRIX,"45.936683389,5.332446671"
7,01009,ANDERT ET CONDON,1300,NaN,ANDERT ET CONDON,"45.787194157,5.657799976"
8,01010,ANGLEFORT,1350,NaN,ANGLEFORT,"45.909091467,5.795056945"
9,01011,APREMONT,1100,NaN,APREMONT,"46.205877513,5.657980818"


In [5]:
communes_df.shape

(38926, 6)

In [6]:
# modification et préparation de la table code_communes_df : suppression colonnes inutiles, en-têtes renommées
communes_df = communes_df.rename(columns = {'code_postal' : 'cp', 'code_commune_insee':'code_commune', 'nom_de_la_commune' : 'commune'})
communes_df = communes_df[['cp', 'code_commune', 'commune']]
communes_df

,cp,code_commune,commune
0,1400,01001,L ABERGEMENT CLEMENCIAT
1,1640,01002,L ABERGEMENT DE VAREY
2,1500,01004,AMBERIEU EN BUGEY
3,1330,01005,AMBERIEUX EN DOMBES
4,1300,01006,AMBLEON
...,...,...,...
38921,20272,2B356,ZALANA
38922,20214,2B361,ZILIA
38923,20272,2B364,ZUANI
38924,20243,2B365,SAN GAVINO DI FIUMORBO


In [7]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
communes_df.isna().sum()

cp              0
code_commune    0
commune         0
dtype: int64

In [8]:
# conversion des données mixtes de la colonne "code commune" en string afin de pouvoir faire le merge
communes_df['code_commune'] = communes_df['code_commune'].astype(str)

In [9]:
#vérifier les types des colonnes : 
communes_df.dtypes

cp               int64
code_commune    object
commune         object
dtype: object

**Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE**
- Source : INSEE - composition communale des zones d'emploi (au format XLSX)
> *Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022. Permet de rattacher chaque commune à sa zone d'emploi, car le taux de chômage actualisé est disponible par zone d'emploi et non par commune*
- URL : https://www.insee.fr/fr/information/4652957

In [10]:
ze_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\ZE2020_au_01-01-2022.xlsx", skiprows = 4, header = 1, sheet_name = 'Composition_communale')
ze_df

,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84
4,01006,Ambléon,8404,Belley,NaN,01,84
...,...,...,...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte,NaN,976,6
34951,97614,Ouangani,601,Mayotte,NaN,976,6
34952,97615,Pamandzi,601,Mayotte,NaN,976,6
34953,97616,Sada,601,Mayotte,NaN,976,6


In [11]:
# modification et préparation de la table code_ze_df : suppression colonnes inutiles, en-têtes renommées
ze_df = ze_df.rename(columns = {"CODGEO":'code_commune2', 
                                          "LIBGEO" : 'commune2',  
                                          "ZE2020" : 'code_ze',
                                         "LIBZE2020" : 'ze'})
ze_df = ze_df[["code_commune2", "commune2", "code_ze", "ze"]]
ze_df

,code_commune2,commune2,code_ze,ze
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404,Belley
...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte
34951,97614,Ouangani,601,Mayotte
34952,97615,Pamandzi,601,Mayotte
34953,97616,Sada,601,Mayotte


In [12]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
ze_df.isna().sum()

code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [13]:
ze_df.dtypes

code_commune2    object
commune2         object
code_ze           int64
ze               object
dtype: object

In [14]:
# conversion des données mixtes de la colonne "code commune" en string afin de pouvoir faire le merge
ze_df['code_commune2'] = ze_df['code_commune2'].astype(str)

In [15]:
#vérification du type des colonnes
ze_df.dtypes

code_commune2    object
commune2         object
code_ze           int64
ze               object
dtype: object

**MERGE sur les 2 premiers JDD CP et codes communes et ZE**
> *Constitution d'un seul jeu avec toutes les informations CP, code commune et nom de la commune, code ZE et nom de la ZE. Dans un second temps, on enlevera les colonnes inutiles (gardées pour vérification).*

> *ATTENTION : le 1er JDD comprenait 39 193 lignes (codes postaux) : suppression depuis Excel des 267 codes postaux 98XXX (Collectivités d’outre-mer non dotées du statut de collectivité territoriale cad non habitées de façon permanente de types : terres australes et antartiques) + MONACO (99) + 5 codes 97XXX non répertoriés > il a donc été réduit à 38926 lignes

> Le second en a uniquement 34 955 (codes communes et codes ZE). Il faudra donc vérifier quels codes postaux n'ont pas de codes INSEE (par hypothèse les CP d'arrondissements à Paris, Marseille, Lyon)

In [16]:
#MERGE des 2 dataframe à partir de la clé code_commune = code_commune2 avec JDD_ze à gauche (34 955) car il y en a moins que dans le JDD_commune (38 931)
INSEE_df = pd.merge(ze_df, communes_df, how = "left", left_on="code_commune2", right_on="code_commune")
print (INSEE_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne : return TRUE where it detects a missing value
INSEE_df.isna().sum()

      code_commune2                 commune2  code_ze                      ze  \
0             01001  L'Abergement-Clémenciat     8405         Bourg en Bresse   
1             01002    L'Abergement-de-Varey     8405         Bourg en Bresse   
2             01004        Ambérieu-en-Bugey     8405         Bourg en Bresse   
3             01005      Ambérieux-en-Dombes     8434  Villefranche-sur-Saône   
4             01006                  Ambléon     8404                  Belley   
...             ...                      ...      ...                     ...   
38867         97614                 Ouangani      601                 Mayotte   
38868         97615                 Pamandzi      601                 Mayotte   
38869         97616                     Sada      601                 Mayotte   
38870         97617                 Tsingoni      601                 Mayotte   
38871         97617                 Tsingoni      601                 Mayotte   

            cp code_commune

code_commune2    0
commune2         0
code_ze          0
ze               0
cp               3
code_commune     3
commune          3
dtype: int64

In [17]:
# identification et visualisation des 3 lignes avec des valeurs manquantes : il s'agit des communes avec arrondissement
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4750,13055,Marseille,9312,Marseille,NaN,NaN,NaN
30158,69123,Lyon,8421,Lyon,NaN,NaN,NaN
32574,75056,Paris,1109,Paris,NaN,NaN,NaN


In [18]:
# Affiche le nombre de valeurs présentes dans chaque colonnes
INSEE_df.count()

code_commune2    38872
commune2         38872
code_ze          38872
ze               38872
cp               38869
code_commune     38869
commune          38869
dtype: int64

In [19]:
#MERGE inversé des 2 dataframeq à partir de la clé code_commune2 = code_commune avec JDD_commune (38 931) à gauche car il en a plus que JDD_ze (34 955) 
INSEE1_df = pd.merge(communes_df, ze_df, how = "left", left_on="code_commune", right_on="code_commune2")
print (INSEE1_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne : return TRUE where it detects a missing value
INSEE1_df.isna().sum()

          cp code_commune                  commune code_commune2  \
0       1400        01001  L ABERGEMENT CLEMENCIAT         01001   
1       1640        01002    L ABERGEMENT DE VAREY         01002   
2       1500        01004        AMBERIEU EN BUGEY         01004   
3       1330        01005      AMBERIEUX EN DOMBES         01005   
4       1300        01006                  AMBLEON         01006   
...      ...          ...                      ...           ...   
38921  20272        2B356                   ZALANA         2B356   
38922  20214        2B361                    ZILIA         2B361   
38923  20272        2B364                    ZUANI         2B364   
38924  20243        2B365   SAN GAVINO DI FIUMORBO         2B365   
38925  20240        2B366                    CHISA         2B366   

                      commune2  code_ze                      ze  
0      L'Abergement-Clémenciat   8405.0         Bourg en Bresse  
1        L'Abergement-de-Varey   8405.0         Bou

cp                0
code_commune      0
commune           0
code_commune2    57
commune2         57
code_ze          57
ze               57
dtype: int64

In [25]:
# identification et visualisation des 57 lignes avec des valeurs manquantes : il s'agit des arrondissements et de certains codes 97
INSEE1_df[INSEE1_df.code_commune2.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13001,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4833,13002,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4834,13003,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4835,13004,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4836,13005,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
4837,13006,13206,MARSEILLE 06,NaN,NaN,NaN,NaN
4838,13007,13207,MARSEILLE 07,NaN,NaN,NaN,NaN
4839,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4840,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4841,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN


In [21]:
#je ne sais pas comment DROPER les lignes 98 donc je vais tout recommencer depuis Excel car ça me saoûle

In [24]:
#maintenant, il faut que j'affecte le code commune et surtout ze au xarrondissements de Marseille / Paris / Lyon à chaque valeur NaN --> pas besoin car le fichier INDICE DE CONCENTRATION D'EMPLOI ne contient pas les arrondissements

In [ ]:
#ensuite, importer le JDD chômage par zone d'emploi : il faudra garder que la colonne CB (2021-T4)

In [ ]:
#importer le JDD indice de concentration d'emploi par commune : droper les lignes et garder que l'année 2018

In [ ]:
#MERGER les 2 JDD avec la base précédente